In [1]:
# # Define initial prompt and output format
# initial_prompt = (
#     "Please look for the following risk factor: Is the client at risk of self-harm? For instance, do they mention suicidal thoughts or ideation? Do they imply they might do physical damage to themselves or to property? Do they reference wanting to 'end it all or say it's 'not worth living'? Please output: 1. risk_type: // suicidality, 2. risk_output: // 'risk present' : this means there is evidence this risk is present in the case 'risk not present' : there is evidence the risk is NOT present or there is no evidence whether the case contains that risk or not. (If in doubt, it is better to err on the side of caution and say 'risk present') 3. explanation: // State words/terms that indicate the reason the risk_output was chosen. Be brief in your explanation. State facts found in the text, do not infer. E.g. 'Client expressed suicidal ideation'. Leave blank for 'risk not present.'"
# )

# output_format_prompt = (
#     "Output should be STRICT JSON, containing: dictionary containing the type of risk with their output and explanation, formatted like this: {'risk_type': 'suicidality', 'risk_output': str, 'explanation': str}'"
# )

initial_prompt = (
        "You are a sentiment analysis classifier. Determine whether the provided text expresses a positive sentiment. "
        "Respond with '1' if it is positive, or '0' if it is negative."
    )

# Output format prompt
output_format_prompt = (
    "You are to act as a binary responder. For every question asked, reply strictly with '1' for positive or '0' for negative. "
    "Do NOT include any additional text or explanation."
)

In [2]:
# # Define output schema
# output_schema = {
#     'key_to_extract': 'risk_output',
#     'value_mapping': {
#         'risk_present': 1,
#         'risk_not_present': 0
#     },
#     'regex_pattern': r'"risk_output":\s*"(.*?)"'
# }

output_schema = {
    'key_to_extract': None,  # Set to None for direct output
    'value_mapping': None,   # Set to None for direct mapping
    'regex_pattern': r'^(0|1)$'  # Match the entire output for binary classification
}

In [3]:
# Set number of optimization iterations
iterations = 3

In [4]:
# Define model providers and models for evaluation and optimization
eval_provider = "ollama"
eval_model = "llama3.1"
optim_provider = "ollama"
optim_model = "llama3.1"

In [5]:
# Path to the CSV file containing review data for evaluation
eval_datapath = "reviews.csv"

------------------------------------------------------------------------------------------

In [6]:
# Import necessary libraries
import pandas as pd
import sys
import os
# Add the parent directory to sys.path
# Use getcwd() to get the current working directory for Jupyter notebooks
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from src.iterative_prompt_optimization import optimize_prompt

In [7]:
# Load and prepare data
eval_data = pd.read_csv(eval_datapath, encoding='ISO-8859-1', usecols=['Text', 'Sentiment'])
eval_data.columns = ['text', 'label']
# Randomly select 50 positive and 50 negative samples
eval_data = (
    eval_data.groupby('label')
    .apply(lambda x: x.sample(n=5, random_state=42))
    .reset_index(drop=True)
)
# Shuffle the DataFrame randomly
eval_data = eval_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Evaluation data shape: {eval_data.shape}")
print(eval_data.head())

Evaluation data shape: (10, 2)
                                                text  label
0  Was this based on a comic-book? A video-game? ...      1
1  If you ask me the first one was really better ...      0
2  When I was a kid, I loved "Tiny Toons". I espe...      1
3  I hate guns and have never murdered anyone, bu...      0
4  I do not have much to say than this is a great...      1


In [8]:
# Run the prompt optimization process
best_prompt, best_metrics = optimize_prompt(
    initial_prompt,
    output_format_prompt,
    eval_data,
    iterations,
    eval_provider=eval_provider,
    eval_model=eval_model,
    optim_provider=optim_provider,
    optim_model=optim_model,
    output_schema=output_schema
)
# After running the optimization process, you can analyze the results by checking 
# the generated log files in the `runs/prompt_optimization_logs_YYYYMMDD_HHMMSS` directory.

Selected evaluation provider: ollama
Selected evaluation model: llama3.1
Selected optimization provider: ollama
Selected optimization model: llama3.1
Estimated token usage: 107190
Estimated cost: $0 API Costs - Running on Local Hardware

Do you want to proceed with the optimization? (Y/N): 
Iteration 1/3
Processing text 1/10
Prediction 1/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 2/10
Prediction 2/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 3/10
Prediction 3/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 4/10
Prediction 4/10: 1 | Ground Truth: 0 ❌ (FP)
Processing text 5/10
Prediction 5/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 6/10
Prediction 6/10: 1 | Ground Truth: 0 ❌ (FP)
Processing text 7/10
Prediction 7/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 8/10
Prediction 8/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 9/10
Prediction 9/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 10/10
Prediction 10/10: 1 | Ground Truth: 1 ✅ (TP)


Evaluation Metrics - Iteration 1
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Metric              ┃  Value ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Precision           │ 0.7143 │
│ Recall              │ 1.0000 │
│ Accuracy            │ 0.8000 │
│ F1-score            │ 0.8333 │
│ Invalid Predictions │      0 │
└─────────────────────┴────────┘


Analyzing misclassifications...


╭──────────────────────────────────────── Analysis of Misclassifications ─────────────────────────────────────────╮
│                                                                                                                 │
│ **Analysis of Misclassifications**                                                                              │
│                                                                                                                 │
│ 1. **Negative (0) texts incorrectly classified as positive:**                                                   │
│         * The first example is a mixed review of the movie "Shuttle". The reviewer expresses both negative and  │
│ positive sentiments, but the LLM model incorrectly classified it as positive.                                   │
│                 + Specific mistakes:                                                                            │
│                         - The text states that the reviewer thinks God would say "Good call" in a hypothetical  │
│ situation, which implies a strong emotional response. However, this is not indicative of a positive review      │
│ overall.                                                                                                        │
│                         - The reviewer criticizes the movie's clichéd plot and lack of originality, which       │
│ should be seen as negative traits.                                                                              │
│                 + Correct classification: Negative (0)                                                          │
│         * The second example is a scathing review of the movie "Dragon Hunt". Despite describing it as one of   │
│ the cheapest action flicks of the eighties, the reviewer still manages to find some positive aspects in it      │
│ being a great party tape.                                                                                       │
│                 + Specific mistakes:                                                                            │
│                         - The text highlights several negative elements of the movie, such as its poor          │
│ production values and ridiculous plot.                                                                          │
│                         - However, the reviewer's tone is more playful than genuinely positive, which should    │
│ have been picked up by the LLM model.                                                                           │
│                                                                                                                 │
│ 2. **Positive (1) texts incorrectly classified as negative:**                                                   │
│         * Unfortunately, there are no examples provided for this category.                                      │
│                                                                                                                 │
│ **Insights into why these errors occurred**                                                                     │
│                                                                                                                 │
│ Based on the analysis above, it appears that the LLM model made mistakes in the following areas:                │
│                                                                                                                 │
│ 1. **Difficulty in detecting nuanced sentiment**: The model struggled to distinguish between mixed reviews and  │
│ genuinely positive ones.                                                                                        │
│ 2. **Sensitivity to strong language**: The use of words like "hate" and "ridiculous" led the model to           │
│ incorrectly classify texts as negative, even when they contained some positive sentiments.                      │
│ 3. **Inability to recognize sarcasm and irony**: The L


Generating new prompt...

Iteration 2/3
Processing text 1/10
Prediction 1/10: 0 | Ground Truth: 1 ❌ (FN)
Processing text 2/10
Prediction 2/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 3/10
Prediction 3/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 4/10
Prediction 4/10: 1 | Ground Truth: 0 ❌ (FP)
Processing text 5/10
Prediction 5/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 6/10
Prediction 6/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 7/10
Prediction 7/10: 1 | Ground Truth: 1 ✅ (TP)
Processing text 8/10
Prediction 8/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 9/10
Prediction 9/10: 0 | Ground Truth: 0 ✅ (TN)
Processing text 10/10
Prediction 10/10: 1 | Ground Truth: 1 ✅ (TP)


...

Here's a revised prompt that incorporates the strategies outlined earlier:

"I'm trying to understand the sentiment behind this text. Please classify it as Positive (1) if it expresses more 
praise than criticism, Negative (0) if it expresses more criticism than praise, and Mixed (-1) if it contains both 
positive and negative sentiments with no clear majority.

When evaluating sentiment, please consider the reviewer's overall opinion, taking into account both explicit 
language and implied tone. Be cautious of strong language, sarcasm, and irony - if you're unsure, choose 'Mixed'. 
Also, acknowledge that some reviews may be nuanced or balanced, praising certain aspects while criticizing others.

Consider factors like reviewer bias, personal preferences, and review style when making your decision. If the text 
uses humor, irony, or sarcasm to express a negative opinion, classify it as Negative (0). Conversely, if the text 
uses strong language but still conveys a positive overall sentiment, classify it as Positive (1).

For cases where the text is balanced or nuanced, with both praise and criticism being significant, choose Mixed 
(-1). This will help you accurately capture the complexity of real-world opinions."

╭────────────────╮
│ Output Format: │
╰────────────────╯

You are to act as a binary responder. For every question asked, reply strictly with '1' for positive or '0' for 
negative. Do NOT include any additional text or explanation.

                         Comparison of All Iterations                         
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Iteration ┃ Precision ┃ Recall ┃ Accuracy ┃ F1-score ┃ Invalid Predictions ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│     1     │    0.7143 │ 1.0000 │   0.8000 │   0.8333 │              0.0000 │
│     2     │    0.8000 │ 0.8000 │   0.8000 │   0.8000 │              0.0000 │
│     3     │    0.8333 │ 1.0000 │   0.9000 │   0.9091 │              0.0000 │
└───────────┴───────────┴────────┴──────────┴──────────┴─────────────────────┘


All logs saved in directory: /Users/danielfiuzadosil/Documents/GitHub/AI-Prompt-Optimiser/runs/prompt_optimization_logs_20240925_155155

Best Prompt:
Here's a revised prompt that incorporates the strategies outlined earlier:

"I'm trying to understand the sentiment behind this text. Please classify it as Positive (1) if it expresses more praise than criticism, Negative (0) if it expresses more criticism than praise, and Mixed (-1) if it contains both positive and negative sentiments with no clear majority.

When evaluating sentiment, please consider the reviewer's overall opinion, taking into account both explicit language and implied tone. Be cautious of strong language, sarcasm, and irony - if you're unsure, choose 'Mixed'. Also, acknowledge that some reviews may be nuanced or balanced, praising certain aspects while criticizing others.

Consider factors like reviewer bias, personal preferences, and review style when making your decision. If the text uses humor, irony, or sarcasm to